In [22]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)
from parquet_data_reader import ParquetDataReader
from models.basic_baseline import MostPopularRecommender


import polars as pl
pl.Config.set_tbl_cols(-1)
import numpy as np
parquet_reader = ParquetDataReader()

In [23]:
train_behavior_df = parquet_reader.read_data("../../data/train/behaviors.parquet")
processed_behavior_df = train_behavior_df.explode("article_ids_clicked")
processed_behavior_df = processed_behavior_df.filter(pl.col("article_ids_clicked").is_not_null())
processed_behavior_df.shape

(234277, 17)

In [24]:
# Initializes the Most popular recommender with the behaviors DataFrame
recommender = MostPopularRecommender(processed_behavior_df)

recommender.fit()

recommendations = recommender.recommend(user_id=123, k=5)
print("Recommended articles:", recommendations)


Recommended articles: shape: (5,)
Series: 'article_ids_clicked' [i32]
[
	9771627
	9771113
	9774532
	9768308
	9770028
]


In [25]:
# Drops duplicates that the same user interacted with the same article multiple times
processed_behavior_df = processed_behavior_df.unique(subset=["user_id", "article_id"])

# Actively displays the shape of the DataFrame after duplicates are dropped
print("Processed behavior DataFrame shape:", processed_behavior_df.shape)

Processed behavior DataFrame shape: (66490, 17)


In [26]:
recommender = MostPopularRecommender(processed_behavior_df)

recommender.fit()

recommendations = recommender.recommend(user_id=123, k=5)
print("Recommended articles:", recommendations)

Recommended articles: shape: (5,)
Series: 'article_ids_clicked' [i32]
[
	9771627
	9771113
	9770028
	9772453
	9772601
]
